# Anansi

Anansi objetiva ser uma ferramenta para análise de mercado e estratégias de operações em mercados, bem como servir como plataforma de crição de trading bots. Diante deste escopo amplo, é inevitável ter de fazer algumas escolhas, os conhecidos "tradeoffs" de qualquer projeto.

## Backtesting

É possível (com pouca programação) estender a classe 'ToStorage' do módulo 'klines' para formar massas de dados para backtesting de qualquer período (claro, respeitando a disponibilidade dos dados de cada corretora...) e de qualquer 'time frame'.

Também não é difícil rodar 'backtestings' que, ao invés de "apontar" para dados em um banco de dados (maneira mais performatica), chama os dados através da web - via API da corretora - em cada ciclo do backtesting.

No entanto, a maneira adotada por padrão é primeiro baixar a maior massa de dados possível (dado um mercado - corretora e símbolo do ativo) do mais refinado time frame (1 minuto para o caso da binance). 

- **Contras**  
  - Alto consumo de tempo para formar a massa de dados;  
  
  - Algum consumo de espaço em disco para armazenamento dos dados mais refinados.  


- **Prós**
  - Possibilidade de criar, sintéticamente, quaisquer outros time frames, mesmo que a corretora não disponibilize alguns dados (interpolação dos dados faltantes);  
  
  - Possibilita rodar backtesting mais dinâmicos, sendo posssivel a rápida comparação de performance dado um mesmo setup aplicado à timeframes diferentes;  
  
  - Maior fidedignadade quando da simulação de preços de entrada/saida dos trades, já que o dado mais refinado tende a traduzir melhor o preço instantâneo ("preço à mercado").

Abaixo os passos para compor a massa de dados mais refinada para o par BTCUSDT da binance:

In [ ]:
from anansi_.marketdata.klines import ToStorage
from anansi_.marketdata.schemas import Market

my_market = Market(broker_name="Binance", ticker_symbol="BTCUSDT")

In [ ]:
btc_to_storage = ToStorage(market=my_market)

In [ ]:
my_raw_klines = btc_to_storage.create_largest_refined_backtesting()

## anansi_.marketdata.klines

Por padrão, retorna as klines a partir da corretora (requer, portanto, conexão com a internet)

In [1]:
from anansi_.marketdata.klines import klines_getter
from anansi_.marketdata.schemas import Market

In [3]:
my_market = Market(broker_name="Binance", quote_symbol="BTC", base_symbol = "USDT")

In [5]:
my_market.ticker_symbol

'BTCUSDT'

Para solicitar as klines, o primeiro passo é usar a função "klines_getter", passando o mercado como argumento. É opcional passar também o "time_frame" desejado o qual, caso não seja passado, será definido como o "time_frame" mais refininado disponível.

In [ ]:
my_klines_getter = klines_getter(my_market)

In [ ]:
my_klines_getter.oldest_open_time()

In [ ]:
type(my_klines_getter)

In [ ]:
my_klines = my_klines_getter.oldest(1457)

As klines são dataframes (da biblioteca pandas) estendidos com alguns métodos, como os indicadores de mercado. Assim, é possível aplicar indicadores diretamente sobre as klines, como abaixo:

In [ ]:
my_klines_getter = klines_getter(my_market)#, time_frame="6h")

In [ ]:
my_klines.apply_indicator.trend.price_from_kline()

In [ ]:
my_klines

In [ ]:
class SetupSMA:
    price_metrics = "hlc3"
    number_samples = 20

my_setup = SetupSMA()

my_klines.apply_indicator.trend.simple_moving_average(my_setup)

In [ ]:
my_klines

## anansi_.marketdata.brokers

###  Exemplo com a Binance

In [ ]:
from anansi_.marketdata.brokers import Binance

my_broker = Binance()

### Alguns atributos da exchange/broker

In [ ]:
my_broker.base_endpoint, my_broker.DateTimeFmt, my_broker.DateTimeUnit

### Verificando horário do servidor com broker.server_time()

In [ ]:
import pendulum # Só para gerar o formato 'human readable', para exemplo

server_time = dict(server_time_timestamp=my_broker.server_time(),
                   server_time_human_readable=(
                       pendulum.from_timestamp(my_broker.server_time()).to_datetime_string()
                   )
                  )

server_time

### Verificando se o limite de requests foi atingido

In [ ]:
my_broker.was_request_limit_reached()

### Finalmente, solicitando algumas klines

In [ ]:
my_klines = my_broker.get_klines(ticker_symbol="BTCUSDT", time_frame="2h")

my_klines

### Função get_broker

É possível repetir todos os passos acima, desde que haja suporte para o broker (passado para a função 'get_broker' pelo parâmetro 'broker_name') instanciando um broker como mostrado abaixo (novamente tomando a binance como exemplo):

In [ ]:
from anansi_.marketdata.brokers import get_broker

my_broker = get_broker(broker_name="binance")

Testando as klines; esteja livre para repetir os passos anteriores :)

In [ ]:
my_klines = my_broker.get_klines(ticker_symbol="BTCUSDT", time_frame="2h")

my_klines

In [ ]:
type(my_klines)